In [1]:
import os
import re
from tqdm import tqdm
from datetime import datetime
import xlrd
import xlutils.copy
import xlwt
from xlwt import Workbook
from copy import deepcopy
import glob

import pandas as pd
import numpy as np
import camelot
import tabula
from pyexcel.cookbook import merge_all_to_a_book

In [2]:
def _getOutCell(outSheet, colIndex, rowIndex):
    """ HACK: Extract the internal xlwt cell representation. """
    row = outSheet._Worksheet__rows.get(rowIndex)
    if not row: return None

    cell = row._Row__cells.get(colIndex)
    return cell

In [3]:
def setOutCell(outSheet, col, row, value):
    """ Change cell value without changing formatting. """
    # HACK to retain cell style.
    previousCell = _getOutCell(outSheet, col, row)
    # END HACK, PART I

    outSheet.write(row, col, value)

    # HACK, PART II
    if previousCell:
        newCell = _getOutCell(outSheet, col, row)
        if newCell:
            newCell.xf_idx = previousCell.xf_idx

In [4]:
def calc_results (df_peak, compound, average_area, constant_1, constant_2, unit):
    base_rrt = float(df_peak['Ret. Time'][df_peak['Name'].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0])
    rrt_master = []
    impurity_master = []
    rrf_master = []
    for index, row in df_peak.iterrows():
        name =row[1]
        if(name.lower() == compound.lower()):
            impurity_master.append(0)
            rrt_master.append(1)
            rrf_master.append(0)
            continue
        if(name == np.nan or name == ''):
            continue
#             impurity_master.append(0)
#             rrt_master.append(0)
            
        area = float(row[3])
        rrf_cond_1 = df_rrf['Compound'].str.contains(compound, flags = re.IGNORECASE)
        rrf_cond_2 = df_rrf['Impurity/Active Name'].str.contains(name, flags = re.IGNORECASE)
        rrf = df_rrf['RRF'][rrf_cond_1 & rrf_cond_2].values.tolist()
        rrt = float(row[2])
        
        if(not(rrf)):
            print("ignoring {}".format(name))
            impurity_master.append(0)
            rrt_res = round(rrt/base_rrt, ndigits=2)
            rrt_master.append(rrt_res)
            rrf_master.append(0)
            continue
            
        rrf = float(rrf[0])
        impurity = round((area/average_area) * constant_1 * constant_2 * (unit/rrf), ndigits=2)
        rrt_res = round(rrt/base_rrt, ndigits=2)
        impurity_master.append(impurity)
        rrt_master.append(rrt_res)
        rrf_master.append(rrf)

    return impurity_master, rrt_master, rrf_master

In [5]:
def shift_row_to_top(df, index_to_shift):
    idx = df.index.tolist()
    idx.remove(index_to_shift)
    df = df.reindex([index_to_shift] + idx)
    return df

In [6]:
def table_extratcor(tables, headers):
    df_result_table =''
    result_tables = []
    for table in tables:
        df_table = table.df
        search = df_table.where(df_table==headers[0]).dropna(how='all').dropna(axis=1)
        inx = list(search.index)
        if(inx):
            inx= inx[0]
            new_header = df_table.iloc[inx]
            new_start_inx = inx+1
            df_table = df_table[new_start_inx:] 
            df_table.columns = new_header
            df_table = df_table[headers]
            result_tables.append(df_table)
        else:
            continue
    df_result_table = pd.concat(result_tables, ignore_index=True)
    return df_result_table

In [7]:
def fill_rs_sheet(output_sheet, df_area_table, df_peak_table, sample_input_list):
    average_area = float(df_area_table["Area"][df_area_table["Title"] == "Average"].values.tolist()[0])
    area_input = list(df_area_table['Area'])
    
    #poject name
    setOutCell(output_sheet, 2, 3, '')
    #Date
    setOutCell(output_sheet, 2, 4, '')
    #Method
    setOutCell(output_sheet, 2, 5, '')
    # WS ID No.
    setOutCell(output_sheet, 1, 9, '')
    # potency
    setOutCell(output_sheet, 3, 9, input_list[-1])
    # use before date
    setOutCell(output_sheet, 5, 9, '')
    # Average area
    setOutCell(output_sheet, 7, 9, average_area)
    # std_wt
    setOutCell(output_sheet, 2, 10, input_list[0])
    #  v1
    setOutCell(output_sheet, 2, 11, input_list[1])
    # v2
    setOutCell(output_sheet, 4, 10, input_list[2])
    #  v3
    setOutCell(output_sheet, 4, 11, input_list[3])
    #  v4
    setOutCell(output_sheet, 6, 10,  input_list[4])
    # v5
    setOutCell(output_sheet, 6, 11, input_list[5])
    # v6
    setOutCell(output_sheet, 8, 10, input_list[6])
    # v7
    setOutCell(output_sheet, 8, 11, input_list[7])
    # factor
    setOutCell(output_sheet, 9, 10, input_list[8])
    # factor
    setOutCell(output_sheet, 9, 11, input_list[9])
    # AR NO
    setOutCell(output_sheet, 1, 14, '')
    # Batch NO
    setOutCell(output_sheet, 3, 14, '')
    # Condition
    setOutCell(output_sheet, 4, 14, '')
    # Label Claim
    setOutCell(output_sheet, 5, 14, sample_input_list[8])
    # per unit
    setOutCell(output_sheet, 7, 14, sample_input_list[9])
    # sample_wt
    setOutCell(output_sheet, 2, 15, sample_input_list[0])
    #  v1
    setOutCell(output_sheet, 2, 16, sample_input_list[1])
    # v2
    setOutCell(output_sheet, 4, 15, sample_input_list[2])
    #  v3
    setOutCell(output_sheet, 4, 16, sample_input_list[3])
    #  v4
    setOutCell(output_sheet, 6, 15, sample_input_list[4])
    # v5
    setOutCell(output_sheet, 6, 16, sample_input_list[5])
    # v6
    setOutCell(output_sheet, 8, 15, sample_input_list[6])
    # v7
    setOutCell(output_sheet, 8, 16, sample_input_list[7])
#     areas
    setOutCell(output_sheet, 12, 5, area_input[0])
    setOutCell(output_sheet, 12, 6, area_input[1])
    setOutCell(output_sheet, 12, 7, area_input[2])
    setOutCell(output_sheet, 12, 8, area_input[3])
    setOutCell(output_sheet, 12, 9, area_input[4])
    setOutCell(output_sheet, 12, 10, area_input[5])
    setOutCell(output_sheet, 12, 11, area_input[6])
    setOutCell(output_sheet, 12, 12, area_input[7])
    setOutCell(output_sheet, 12, 13, area_input[8])
#   Impurity table  
    table_row = 20
    for index, row in df_peak_table.iterrows():
        if(table_row > 60):
            break
        setOutCell(output_sheet, 1, table_row, row[0])
        setOutCell(output_sheet, 2, table_row, row[1])
        setOutCell(output_sheet, 3, table_row, row[2])
        setOutCell(output_sheet, 4, table_row, row[3])
        setOutCell(output_sheet, 5, table_row, row[4])
        setOutCell(output_sheet, 6, table_row, row[5])
        table_row +=1

    sum_of_impurities = round(df_peak_table["% w/w"].sum(), ndigits=2)
    setOutCell(output_sheet, 6, 62, sum_of_impurities)


In [8]:
def initiate_report_creation(compound, input_lits):
    sample_wt = df_sample_prep['vials'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    sample_v1 = df_sample_prep['v1'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    sample_v2 = df_sample_prep['v2'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    sample_v3 = df_sample_prep['v3'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    sample_v4 = df_sample_prep['v4'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    sample_v5 = df_sample_prep['v5'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    sample_v6 = df_sample_prep['v6'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    sample_v7 = df_sample_prep['v7'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    label_claim = df_sample_prep['label claim'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    unit = df_sample_prep['per unit'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]

    constant_1 = (input_list[0]/input_list[1]) * (input_list[2]/input_list[3]) * (input_list[4]/input_list[5])*(input_list[6]/input_list[7]) * (input_list[8]/input_list[9])
    constant_2 = (sample_v1/sample_wt) * (sample_v3/sample_v2) * (sample_v5/sample_v4) * (sample_v7/sample_v6) * (input_list[10]/label_claim)

    # peak table extratcion
    tables = camelot.read_pdf(chrom_input, pages= 'all',flavor='stream')
    df_peak_table = table_extratcor(tables, chrom_headers)
    inx_to_shift = df_peak_table[df_peak_table["Name"].str.contains(compound, flags = re.IGNORECASE)].index[0]
    df_peak_table = shift_row_to_top(df_peak_table, inx_to_shift)
    cond_1 = df_peak_table["Name"] == ''
    cond_2 = df_peak_table["Name"] == np.nan
    inxs_to_remove = df_peak_table[cond_1 | cond_2].index
    df_peak_table = df_peak_table.drop(inxs_to_remove)

    # area table extraction
    tables = camelot.read_pdf(area_input, pages= 'all',flavor='stream')
    df_area_table = table_extratcor(tables, area_headers)
    df_area_table = df_area_table[['Title', 'Area']]
    average_area = float(df_area_table["Area"][df_area_table["Title"] == "Average"].values.tolist()[0])

    # impurity calculation
    impurities, rrts, rrfs = calc_results(df_peak_table, compound, average_area, constant_1, constant_2, unit)
    df_peak_table['RRT'] = rrts
    df_peak_table['RRF'] = rrfs
    df_peak_table["% w/w"] = impurities
    df_peak_table = df_peak_table[['Name', 'Ret. Time','RRT', 'RRF', 'Area', '% w/w']]

    # writing to output sheet
    sample_input_list = [sample_wt, sample_v1, sample_v2, sample_v3, sample_v4, sample_v5, sample_v6, sample_v7, label_claim, unit]
    fill_rs_sheet(rs_template_sheet, df_area_table, df_peak_table, sample_input_list)

    

In [ ]:
# Bumetanide
# Acyclovir
# Famotidine
# ketorolacTromethamine
# LabetalolHCl
# input_list = [20.48,20,1,100,5,20,1,1,1,1,99.0]

# compound = input("Enter the compund name [As mentioned in the chromatogram] ")
compound = 'Acyclovir'
input_list = [50.43,100,5,50,5,50,1,1,1,1,94.4]
# input data sources
df_rrf = pd.read_excel(os.path.join(os.getcwd(), 'data', 'Templates', 'RRF-template.xlsx'))
df_sample_prep = pd.read_excel(os.path.join(os.getcwd(), 'data', 'Templates', 'Sample Preparation.xlsx'))
chrom_input = os.path.join(os.getcwd(), "data", "chromatograms", "{}-chromatogram.pdf".format(compound))
area_input = os.path.join(os.getcwd(), "data", "Areas", "{}-areas.pdf".format(compound))

# input_list = [0]*11
# input_list[0] = float(input("Enter the Weight taken "))
# input_list[1] = float(input("Enter the standard preparation v1 ")) 
# input_list[2] = float(input("Enter the standard preparation v2 ")) 
# input_list[3] = float(input("Enter the standard preparation v3 ")) 
# input_list[4] = float(input("Enter the standard preparation v4 ")) 
# input_list[5] = float(input("Enter the standard preparation v5 ")) 
# input_list[6] = float(input("Enter the standard preparation v6 ")) 
# input_list[7] = float(input("Enter the standard preparation v7 ")) 
# input_list[8] = float(input("Enter the standard preparation factor 1 ")) 
# input_list[9] = float(input("Enter the standard preparation factor 2 ")) 
# input_list[10] = float(input("Enter the standard preparation Potency ")) 
 

chrom_headers = ['Peak#','Name','Ret. Time','Area','Area%','RRT']
area_headers = ['Title', 'Ret. Time', 'Area', 'Area%', 'NTP', 'Tailing Factor']



# Output sheet
rs_template_input = xlrd.open_workbook(os.path.join(os.getcwd(), "data", "Templates",'RS-template.xls'), formatting_info=True)
rs_template = xlutils.copy.copy(rs_template_input)
rs_template_sheet = rs_template.get_sheet(0)

initiate_report_creation(compound, input_list)
rs_template.save(os.path.join(os.getcwd(), "data", 'output', '{}-RS.xls'.format(compound)))
print("Report saved successfully, check Output folder.")

In [ ]:
headers = ['Name','Ret. Time','Area']
headers = ['Area']

In [ ]:
chrom_input = os.path.join(os.getcwd(), "data", "chromatograms", "Famotidine-chromatogram.pdf".format(compound))
area_input = os.path.join(os.getcwd(), "data", "areas", "Acyclovir-areas.pdf".format(compound))
tables = camelot.read_pdf(chrom_input, pages= 'all',flavor='stream')

In [ ]:
df_result_table =''
result_tables = []
for table in tables:
    df_table = table.df
    search = df_table.where(df_table==headers[0]).dropna(how='all').dropna(axis=1)
    inx = list(search.index)
    if(inx):
        inx= inx[0]
        new_header = df_table.iloc[inx]
        new_start_inx = inx+1
        df_table = df_table[new_start_inx:] 
        df_table.columns = new_header
        df_table = df_table[headers]
        result_tables.append(df_table)
    else:
        continue
df_result_table = pd.concat(result_tables, ignore_index=True)

In [ ]:
df_result_table = df_result_table.drop_duplicates(keep='first')
df_peak_table = df_result_table
inx_to_shift = df_peak_table[df_peak_table["Name"].str.contains(compound, flags = re.IGNORECASE)].index[0]
df_peak_table = shift_row_to_top(df_peak_table, inx_to_shift)
cond_1 = df_peak_table["Name"] == ''
cond_2 = df_peak_table["Name"] == np.nan
inxs_to_remove = df_peak_table[cond_1 | cond_2].index
df_peak_table = df_peak_table.drop(inxs_to_remove)


In [ ]:
df_peak_table

In [41]:
compound = 'Vancomycin-B'
headers = ['Name','Ret. Time','Area']
# headers = ['Area']
chrom_input = os.path.join(os.getcwd(), "data", "RS-area-norm", '1043_006_40_75_6M_I_SPL_01.pdf')
tables = camelot.read_pdf(chrom_input, pages= 'all')
# tables = camelot.read_pdf(chrom_input, pages= 'all', flavour = 'stream')
# df_peak_table = table_extratcor(tables, headers)
# df_peak_table = df_peak_table.drop_duplicates(keep="first")
# base_rt = float(df_peak_table['Ret. Time'][df_peak_table["Name"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0])
# compound_row = df_peak_table[df_peak_table["Name"].str.contains(compound, flags = re.IGNORECASE)].index
# df_peak_table = df_peak_table.drop(compound_row)
# cond_1 = df_peak_table["Name"] == ''
# cond_2 = df_peak_table["Name"] == np.nan
# inxs_to_remove = df_peak_table[cond_1 | cond_2].index
# df_peak_table = df_peak_table.drop(inxs_to_remove)

In [20]:
print(len(tables))

4


In [55]:
df_table = tables[0].df
# df_table = tables[0].df.T.reindex(index=df_table.index[::-1])

In [56]:
df_table

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,12,11,10,9,8,7,6,5,4,3,2,1,Peak#
1,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Name
2,5.44,5.08,4.72,4.42,4.20,3.88,3.37,3.00,2.93,2.85,2.69,2.56,Ret. Time
3,20574,11139,31937,61148,11810,33125,1234,2109,6133,2436,24187,843,Area
4,0.04,0.02,0.06,0.11,0.02,0.06,0.00,0.00,0.01,0.00,0.04,0.00,Area%
5,0.44,0.41,0.38,0.36,0.34,0.31,0.27,0.24,0.24,0.23,0.22,0.21,RRT


In [61]:
df_result_table =''
result_tables = []
for table in tables[0:1]:
    df_table = table.df
    search = df_table.where(df_table==headers[0]).dropna(how='all').dropna(axis=1)
    inx = list(search.index)
    if(inx):
        print(table)
        inx= inx[0]
        new_header = df_table.iloc[inx]
        print(new_header)
        new_start_inx = inx+1
        df_table = df_table[new_start_inx:]
        df_table.columns = new_header
        df_table = df_table[headers]
        result_tables.append(df_table)
    else:
        continue

<Table shape=(6, 13)>
0     Unknown
1     Unknown
2     Unknown
3     Unknown
4     Unknown
5     Unknown
6     Unknown
7     Unknown
8     Unknown
9     Unknown
10    Unknown
11    Unknown
12       Name
Name: 1, dtype: object


KeyError: "['Area', 'Ret. Time'] not in index"

In [49]:
compound = 'Ketorolac Tromethamine'
area_inputs = glob.glob(os.path.join(os.getcwd(), "data", "Imp-vs-imp", compound, '1008-019A_40°C 75% RH 1.5M(U)'))
area_headers = ['Name', 'Area']
area_tables =  []
for area_input in area_inputs:
    tables = camelot.read_pdf(area_input, pages= 'all',flavor='stream')
    df_area = table_extratcor(tables, area_headers)
    df_area = df_area[['Name','Area']]
    area_tables.append(df_area)
df_area_table = pd.concat(area_tables)

In [58]:
df_area_table = df_area_table.drop(df_area_table[df_area_table['Name'] == ''].index)
df_area_table["Area"] = df_area_table["Area"].astype(float)

In [69]:
round(df_area_table['Area'][df_area_table['Name'] == 'Related Compound-A'].mean())

17340

In [70]:
set(df_area_table['Name'])

{'Ketorolac Tromethamine',
 'Related Compound-A',
 'Related Compound-B',
 'Related Compound-C',
 'Related Compound-D'}

In [18]:
compound = 'Ropivacaine HCl'
a_date = '04.01.22'
year = year = str(datetime.today().year)
area_input = os.path.join(os.getcwd(), "data", year, compound, "RS", a_date, "{}-areas.pdf".format(compound))
chrom_inputs = glob.glob(os.path.join(os.getcwd(), "data", year, compound, "RS", a_date, '*.pdf'))
chrom_inputs.remove(area_input)

In [19]:
area_input

'D:\\projects\\document-processing\\Impurity-calculator\\data\\2022\\Ropivacaine HCl\\RS\\04.01.22\\Ropivacaine HCl-areas.pdf'

In [20]:
chrom_inputs

['D:\\projects\\document-processing\\Impurity-calculator\\data\\2022\\Ropivacaine HCl\\RS\\04.01.22\\10030-010A 40°C-75%RH_2M(I)_ARR_1030_21_089.pdf',
 'D:\\projects\\document-processing\\Impurity-calculator\\data\\2022\\Ropivacaine HCl\\RS\\04.01.22\\10030-010B 40°C-75%RH_2M(I)_ARR_1030_21_090.pdf',
 'D:\\projects\\document-processing\\Impurity-calculator\\data\\2022\\Ropivacaine HCl\\RS\\04.01.22\\10030-011A 40°C-75%RH_2M(I)_ARR_1030_21_091.pdf',
 'D:\\projects\\document-processing\\Impurity-calculator\\data\\2022\\Ropivacaine HCl\\RS\\04.01.22\\10030-011B 40°C-75%RH_2M(I)_ARR_1030_21_092.pdf',
 'D:\\projects\\document-processing\\Impurity-calculator\\data\\2022\\Ropivacaine HCl\\RS\\04.01.22\\10030-013A 40°C-75%RH_1.5M(I)_ARR_1031_21_093.pdf',
 'D:\\projects\\document-processing\\Impurity-calculator\\data\\2022\\Ropivacaine HCl\\RS\\04.01.22\\10030-013B 40°C-75%RH_1.5M(I)_ARR_1031_21_094.pdf',
 'D:\\projects\\document-processing\\Impurity-calculator\\data\\2022\\Ropivacaine HCl\\RS\

In [24]:
d = pd.DataFrame()

In [27]:
d.empty()

TypeError: 'bool' object is not callable

In [ ]:
SELECT `referrals`.`id`, `referrals`.`doctor_officer_id`, `referrals`.`internal_doctor_id`, 
`referrals`.`referred_by_id`, `referrals`.`actual_internal_doctor_id`, `referrals`.`status`, 
`referrals`.`patient_name`, `referrals`.`country_id`, `referrals`.`patient_mobile`, `referrals`.`visit_date`,
`referrals`.`actual_visit_date`, `referrals`.`yh_number`, `referrals`.`ip_number`, `referrals`.`referral_type`, 
`referrals`.`admitted_by_id`, `referrals`.`validated_by_id`, `referrals`.`valid`, `referrals`.`pic_1`, 
`referrals`.`pic_2`, `referrals`.`pic_3`, `referrals`.`pic_4`, `referrals`.`pic_5`, `referrals`.`executive_pic_1`,
`referrals`.`executive_pic_2`, `referrals`.`executive_pic_3`, `referrals`.`executive_pic_4`, 
`referrals`.`executive_pic_5`, `referrals`.`admitted_at`, `referrals`.`created_at`, `referrals`.`updated_at`, 
`referrals`.`deleted_at`, `referrals`.`patient_location`, `referrals`.`he_remark`, `referrals`.`ppd_valid_remark`, 
`referrals`.`rejection_reason`, `referrals`.`ppd_invalid_remark`, `referrals`.`user_remark`, 
`locations`.`id` AS 'locations_id', `locations`.`name`, `locations`.`type`, `locations`.`address`, 
`locations`.`active`, `locations`.`referral`, `locations`.`lat`, `locations`.`lng`, `locations`.`created_by_id`, 
`locations`.`created_at` AS 'locations_created_at', `locations`.`updated_at` AS 'locations_updated_at',
`locations`.`deleted_at` AS 'locations_deleted_at', `locations`.`locality`, `locations`.`city`, `locations`.`state`,
`locations`.`country`, `locations`.`address_parts`, `doctorofficer`.`id` AS 'doctorofficer_id', 
`doctorofficer`.`company`, `doctorofficer`.`changed_company`, `doctorofficer`.`doctor_code`, 
`doctorofficer`.`changed_doctor_code`, `doctorofficer`.`turf`, `doctorofficer`.`changed_turf`, 
`doctorofficer`.`designation`, `doctorofficer`.`changed_designation`, `doctorofficer`.`specialization_id`, 
`doctorofficer`.`changed_specialization_id`, `doctorofficer`.`type` AS 'doctorofficer_type', 
`doctorofficer`.`name` AS 'doctorofficer_name', `doctorofficer`.`changed_name`, 
`doctorofficer`.`country_id` AS 'doctorofficer_country_id', `doctorofficer`.`changed_country_id`, 
`doctorofficer`.`mobile`, `doctorofficer`.`changed_mobile`, `doctorofficer`.`address` AS 'doctorofficer_address', 
`doctorofficer`.`locality` AS 'doctorofficer_locality', `doctorofficer`.`changed_locality`, 
`doctorofficer`.`city` AS 'doctorofficer_city', `doctorofficer`.`changed_city`, 
`doctorofficer`.`state` AS 'doctorofficer_state', `doctorofficer`.`changed_state`, 
`doctorofficer`.`country` AS 'doctorofficer_country', `doctorofficer`.`changed_country`, 
`doctorofficer`.`pincode`, `doctorofficer`.`changed_pincode`, `doctorofficer`.`changed_address`,
`doctorofficer`.`address_parts` AS 'doctorofficer_address_parts', `doctorofficer`.`status` AS 'doctorofficer_status',
`doctorofficer`.`active` AS 'doctorofficer_active', `doctorofficer`.`lat` AS 'doctorofficer_lat',
`doctorofficer`.`changed_lat`, `doctorofficer`.`lng` AS 'doctorofficer_lng', `doctorofficer`.`changed_lng`, 
`doctorofficer`.`pic_1` AS 'doctorofficer_pic_1', `doctorofficer`.`changed_pic_1`,
`doctorofficer`.`pic_2` AS 'doctorofficer_pic_2', `doctorofficer`.`changed_pic_2`, 
`doctorofficer`.`pic_3` AS 'doctorofficer_pic_3', `doctorofficer`.`changed_pic_3`, `doctorofficer`.`user_id`, 
`doctorofficer`.`approved_by_id`, `doctorofficer`.`approved_date`, `doctorofficer`.`changed_by_id`, 
`doctorofficer`.`changed_at`, `doctorofficer`.`created_at` AS 'doctorofficer_created_at', 
`doctorofficer`.`updated_at` AS 'doctorofficer_updated_at', `doctorofficer`.`deleted_at` AS 'doctorofficer_deleted_at', 
`categories`.`id` AS 'categories_id', `categories`.`location_id`, 
`categories`.`doctor_officer_id` AS 'categories_doctor_officer_id', `categories`.`category_id`, 
`categories`.`changed_category_id`, `categories`.`created_at` AS 'categories_created_at', 
`categories`.`updated_at` AS 'categories_updated_at', `categories`.`deleted_at` AS 'categories_deleted_at', 
`category`.`id` AS 'category_id_0', `category`.`name` AS 'category_name', `category`.`created_at` AS 'category_created_at', 
`category`.`updated_at` AS 'category_updated_at', `category`.`deleted_at` AS 'category_deleted_at' 
FROM `referrals` LEFT OUTER JOIN `referrals_reporting_locations` 
ON (`referrals_reporting_locations`.`referral_id` = `referrals`.`id`) 
LEFT OUTER JOIN `reporting_locations` AS 'locations' ON (`locations`.`id` = `referrals_reporting_locations`.`location_id`)
LEFT OUTER JOIN `doctors_officers` AS 'doctorofficer' ON (`doctorofficer`.`id` = `referrals`.`doctor_officer_id`) 
LEFT OUTER JOIN `doctors_officers_categories` AS 'categories' ON (`categories`.`doctor_officer_id` = `doctorofficer`.`id`)
LEFT OUTER JOIN `categories` AS 'category' ON (`category`.`id` = `categories`.`category_id`) 
WHERE ((`referrals`.`deleted_at` IS NULL) AND (`referrals`.`valid` = 't') AND (`referrals`.`status` = 'a') 
       AND (`referrals`.`created_at` >= '2021-12-25 18:30:00.000000') 
       AND (`referrals`.`created_at` <= '2022-01-25 18:30:00.000000') 
       AND (`categories`.`location_id` = `referrals_reporting_locations`.`location_id`) 
       AND (`doctorofficer`.`type` = 'd') AND (`doctorofficer`.`active` = 1))
GROUP BY `referrals`.`id`